# Using Hypothetical Document Embeddinga (HyDE) to Improve Retrieval

> 📚 This cookbook has an accompanying article with a complete walkthrough ["Optimizing Retrival with HyDE"](https://haystack.deepset.ai/blog/optimizing-retrieval-with-hyde)

In this coookbook, we are building Haystack components that allow us to easily incorporate HyDE into our RAG pipelines, to optimize retrieval.

> To learn more about HyDE and when it's useful, check out our [guide to Hypothetical Document Embeddings (HyDE)](https://docs.haystack.deepset.ai/v2.0/docs/hypothetical-document-embeddings-hyde)

## Install Requirements

In [1]:
# !pip install haystack-ai sentence-transformers datasets

In the following sections, we will be using the `OpenAIGenerator`, so we need to provide our API key 👇

In [2]:
from getpass import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass("Enter your openAI key:")

## Building a Pipeline for Hypothetical Document Embeddings

We will build a Haystack pipeline that generates 'fake' documents.
For this part, we are using the `OpenAIGenerator` with a `PromptBuilder` that instructs the model to generate paragraphs.

In [3]:
from haystack.components.generators.openai import OpenAIGenerator
from haystack.components.builders import PromptBuilder

generator = OpenAIGenerator(
    model="gpt-3.5-turbo",
    generation_kwargs={"n": 5, "temperature": 0.75, "max_tokens": 400},
)

template="""Given a question, generate a paragraph of text that answers the question.
            Question: {{question}}
            Paragraph:"""

prompt_builder = PromptBuilder(template=template)

Next, we use the `OutputAdapter` to transform the generated paragraphs into a List of Documents. This way, we will be able to use the `SentenceTransformersDocumentEmbedder` to create embeddings, since this component expects `List[Document]`

Components that use heavy resources, like LLMs or embedding models, additionally have a warm_up() method. When you run a component like this on its own, you must run warm_up() after initializing it, but before running it, like this:

In [4]:
from haystack import Document
from haystack.components.converters import OutputAdapter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from typing import List

adapter = OutputAdapter(
    template="{{answers | build_doc}}",
    output_type=List[Document],
    custom_filters={"build_doc": lambda data: [Document(content=d) for d in data]}
)

embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")#First, initialize the component
embedder.warm_up()# Then, warm it up to load the model

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\sri.karan\.conda\envs\rag\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sri.karan\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Finally, we create a custom component, `HypotheticalDocumentEmbedder`, that expects `documents` and can return a list of `hypotethetical_embeddings` which is the average of the embeddings from the "hypothetical" (fake) documents. To learn more about this technique and where it's useful, check out our [Guide to HyDE](https://docs.haystack.deepset.ai/v2.0/docs/hypothetical-document-embeddings-hyde)

In [5]:
from numpy import array, mean
from haystack import component

@component
class HypotheticalDocumentEmbedder:

  @component.output_types(hypothetical_embedding=List[float])
  def run(self, documents: List[Document]):
    stacked_embeddings = array([doc.embedding for doc in documents])
    avg_embeddings = mean(stacked_embeddings, axis=0)
    hyde_vector = avg_embeddings.reshape((1, len(avg_embeddings)))
    return {"hypothetical_embedding": hyde_vector[0].tolist()}

We add all of our components into a pipeline to genereate a hypothetical document embedding 🚀👇

In [6]:
from haystack import Pipeline

hyde = HypotheticalDocumentEmbedder()

pipeline = Pipeline()
pipeline.add_component(name="prompt_builder", instance=prompt_builder)
pipeline.add_component(name="generator", instance=generator)
pipeline.add_component(name="adapter", instance=adapter)
pipeline.add_component(name="embedder", instance=embedder)
pipeline.add_component(name="hyde", instance=hyde)

pipeline.connect("prompt_builder", "generator")
pipeline.connect("generator.replies", "adapter.answers")
pipeline.connect("adapter.output", "embedder.documents")
pipeline.connect("embedder.documents", "hyde.documents")

query = "What should I do if I have a fever?"
result = pipeline.run(data={"prompt_builder": {"question": query}})

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
print(result["hyde"])

{'hypothetical_embedding': [0.0034947479609400032, 0.03057028278708458, 0.03657022649422288, 0.04142231792211533, 0.06088000908493996, 0.010732880351133645, 0.0015576664241962134, 0.039151085913181304, 0.019605993758887053, -0.02549406196922064, -0.08347966894507408, -0.04400303736329079, 0.07709210589528084, 0.0879963144659996, 0.03589583393186331, -0.0796284593641758, 0.018148287758231164, 0.005981175787746906, -0.003148116637021303, -0.0267334271222353, -0.1018729031085968, -0.02116571506485343, -0.023622474120929837, 0.03710565268993378, -0.024143813364207744, 0.10136220008134841, 0.03964416553499177, 0.027055959217250347, -0.003908039559610188, 0.029470601864159108, 0.007206507958471775, -0.03363596126437187, 0.007229621731676161, -0.03701051119714975, -0.019760082056745887, 0.07434764429926873, -0.024239076115190984, -0.023802184872329234, -0.04384289793670178, 0.044565974920988086, 0.0099062564317137, 0.10630172193050384, 0.00034430441446602343, -0.023563247919082642, 0.09104609

## Build a HyDE Component That Encapsulates the Whole Logic

This section shows you how to create a `HypotheticalDocumentEmbedder` that instead, encapsulates the entire logic, and also allows us to provide the embedding model as an optional parameter.

This "mega" components does a few things:

- Allows the user to pick the LLM which generates the hypothetical documents
- Allows users to define how many documents should be created with `nr_completions`
- Allows users to define the embedding model they want to use to generate the HyDE embeddings.



In [8]:
from haystack import Pipeline, component, Document, default_to_dict, default_from_dict
from haystack.components.converters import OutputAdapter
from haystack.components.embedders.sentence_transformers_document_embedder import SentenceTransformersDocumentEmbedder
from haystack.components.generators.openai import OpenAIGenerator
from haystack.components.builders import PromptBuilder

from typing import Dict, Any, List
from numpy import array, mean
from haystack.utils import Secret


@component
class HypotheticalDocumentEmbedder:

    def __init__(
        self,
        instruct_llm: str = "gpt-3.5-turbo",
        instruct_llm_api_key: Secret = Secret.from_env_var("OPENAI_API_KEY"),
        nr_completions: int = 5,
        embedder_model: str = "sentence-transformers/all-MiniLM-L6-v2",
    ):
        self.instruct_llm = instruct_llm
        self.instruct_llm_api_key = instruct_llm_api_key
        self.nr_completions = nr_completions
        self.embedder_model = embedder_model
        self.generator = OpenAIGenerator(
            api_key=self.instruct_llm_api_key,
            model=self.instruct_llm,
            generation_kwargs={"n": self.nr_completions, "temperature": 0.75, "max_tokens": 400},
        )
        self.prompt_builder = PromptBuilder(
            template="""Given a question, generate a paragraph of text that answers the question.
            Question: {{question}}
            Paragraph:
            """
        )

        self.adapter = OutputAdapter(
            template="{{answers | build_doc}}",
            output_type=List[Document],
            custom_filters={"build_doc": lambda data: [Document(content=d) for d in data]},
        )

        self.embedder = SentenceTransformersDocumentEmbedder(model=embedder_model, progress_bar=False)
        self.embedder.warm_up()

        self.pipeline = Pipeline()
        self.pipeline.add_component(name="prompt_builder", instance=self.prompt_builder)
        self.pipeline.add_component(name="generator", instance=self.generator)
        self.pipeline.add_component(name="adapter", instance=self.adapter)
        self.pipeline.add_component(name="embedder", instance=self.embedder)
        self.pipeline.connect("prompt_builder", "generator")
        self.pipeline.connect("generator.replies", "adapter.answers")
        self.pipeline.connect("adapter.output", "embedder.documents")

    def to_dict(self) -> Dict[str, Any]:
        data = default_to_dict(
            self,
            instruct_llm=self.instruct_llm,
            instruct_llm_api_key=self.instruct_llm_api_key,
            nr_completions=self.nr_completions,
            embedder_model=self.embedder_model,
        )
        data["pipeline"] = self.pipeline.to_dict()
        return data

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> "HypotheticalDocumentEmbedder":
        hyde_obj = default_from_dict(cls, data)
        hyde_obj.pipeline = Pipeline.from_dict(data["pipeline"])
        return hyde_obj

    @component.output_types(hypothetical_embedding=List[float])
    def run(self, query: str):
        result = self.pipeline.run(data={"prompt_builder": {"question": query}})
        # return a single query vector embedding representing the average of the hypothetical document embeddings
        stacked_embeddings = array([doc.embedding for doc in result["embedder"]["documents"]])
        avg_embeddings = mean(stacked_embeddings, axis=0)
        hyde_vector = avg_embeddings.reshape((1, len(avg_embeddings)))
        return {"hypothetical_embedding": hyde_vector[0].tolist()}

## Use HyDE For Retrieval

Let's see how we can use this component in a full pipeline. First, let's index some documents into an `InMemoryDocumentStore`

In [9]:
from datasets import load_dataset, Dataset

from haystack import Pipeline, Document
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.writers import DocumentWriter
from haystack.document_stores.in_memory import InMemoryDocumentStore

embedder_model = "sentence-transformers/all-MiniLM-L6-v2"


def index_docs(data: Dataset):
    # create a data store and indexing pipeline with the components
    document_store = InMemoryDocumentStore()

    pipeline = Pipeline()
    pipeline.add_component("cleaner", DocumentCleaner())
    pipeline.add_component("splitter", DocumentSplitter(split_by="sentence", split_length=10))
    pipeline.add_component("embedder", SentenceTransformersDocumentEmbedder(model=embedder_model))
    pipeline.add_component("writer", DocumentWriter(document_store=document_store, policy="skip"))

    # connect the components
    pipeline.connect("cleaner", "splitter")
    pipeline.connect("splitter", "embedder")
    pipeline.connect("embedder", "writer")

    # index the documents and return the data store
    pipeline.run({"cleaner": {"documents": [Document.from_dict(doc) for doc in data["train"]]}})
    return document_store


data = load_dataset("Tuana/game-of-thrones")
doc_store = index_docs(data)

Generating train split:   0%|          | 0/2357 [00:00<?, ? examples/s]

Batches:   0%|          | 0/103 [00:00<?, ?it/s]

We can now run a retrieval pipeline that doesn't just retrieve based on the query embeddings, instead, it uses the `HypotheticalDocumentEmbedder` to create hypothetical document embeddings based on our `query` and uses these new embeddings to retrieve documents.

In [10]:
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

def retriever_with_hyde(doc_store):
    hyde = HypotheticalDocumentEmbedder(instruct_llm="gpt-3.5-turbo", nr_completions=5)
    retriever = InMemoryEmbeddingRetriever(document_store=doc_store)

    retrieval_pipeline = Pipeline()
    retrieval_pipeline.add_component(instance=hyde, name="query_embedder")
    retrieval_pipeline.add_component(instance=retriever, name="retriever")

    retrieval_pipeline.connect("query_embedder.hypothetical_embedding", "retriever.query_embedding")
    return retrieval_pipeline

retrieval_pipeline = retriever_with_hyde(doc_store)
query = "Who is Araya Stark?"
retrieval_pipeline.run(data={"query_embedder": {"query": query}, "retriever": {"top_k": 5}})

{'retriever': {'documents': [Document(id=f21283f81311acf8c402cc0ef79a1fcc1d583fdfade0f48939c08d2242a44985, content: ''''Arya Stark''' is a fictional character in American author George R. R. Martin's ''A Song of Ice a...', meta: {'name': '43_Arya_Stark.txt', 'source_id': '557ad8704d47b0e12aef859563c87c86782b832f3d06909ede1c0e36dc039540', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0}, score: 0.7199782127335106),
   Document(id=097bcaeffac04eaa9f97320bb9c15a692e95a427a86f60270abb18c079ebd867, content: '=== Sansa Stark ===
   Sansa Stark is the second child and elder daughter of Eddard and Catelyn Stark. ...', meta: {'name': '30_List_of_A_Song_of_Ice_and_Fire_characters.txt', 'source_id': '0c945854b1eb4db6cde79229fb608b5d22933274396a52b0fb0c1945e8fcde36', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0}, score: 0.7009547645339009),
   Document(id=a89c0e8f7acd10f8e7b87577f00287f4f82d78e316e2c32f25ffedaafaae8f10, content: '===Arya Stark===
   '''Arya Stark''' portrayed by Mai